<a href="https://colab.research.google.com/github/ritwikaswec1240/email-extractor/blob/main/Email_Id_Extractor_from_sites_in_Scrapy_Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Email Extractor from Websites
This project extracts email addresses from webpages using Python (requests, BeautifulSoup, regex).


In [ ]:
!pip install scrapy scrapy-selenium
!apt-get update
!apt-get install -y chromium chromium-driver


In [ ]:
!apt-get update -q
!apt-get install -y chromium-browser chromium-chromedriver

In [ ]:
import shutil
CHROMEDRIVER_PATH = shutil.which("chromedriver")
print("ChromeDriver Path:", CHROMEDRIVER_PATH)


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy_selenium import SeleniumRequest
import re

class EmailSpider(scrapy.Spider):
    name = "emailspider"
    custom_settings = {
        "FEEDS": {"emails.csv": {"format": "csv"}},  # save results to CSV
        "SELENIUM_DRIVER_NAME": "chrome",
        "SELENIUM_DRIVER_EXECUTABLE_PATH": "/usr/bin/chromedriver",
        "SELENIUM_DRIVER_ARGUMENTS": ["--headless", "--no-sandbox", "--disable-dev-shm-usage"],
        "LOG_LEVEL": "INFO",
    }

    def start_requests(self):
        yield SeleniumRequest(
            url="https://www.geeksforgeeks.org/",  # change this to your target site
            wait_time=5,
            callback=self.parse
        )

    def parse(self, response):
        page_url = response.url
        emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", response.text)

        for email in set(emails):
            yield {"page_url": page_url, "email": email}

        # ✅ follow internal links safely
        for link in response.css("a::attr(href)").getall():
            if link and link.startswith("https://www.geeksforgeeks.org"):
                yield SeleniumRequest(
                    url=link,
                    wait_time=3,
                    callback=self.parse
                )

# ========================
# Run the spider
# ========================
process = CrawlerProcess()
process.crawl(EmailSpider)
process.start()
